In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import MySQLdb as db
import credentials
import pickle
import time
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [83]:
%%time
#Establish connection to the MySQLDB
MySQLDB = db.connect(credentials.host, credentials.user, credentials.password, credentials.db)

Wall time: 82.4 ms


In [84]:
# Standardized shipping methods based primarily upon what is selectable through the FedEx API here:
# https://www.fedex.com/ratefinder/home. 'Home Delivery' and 'Smartpost' are not selectable.
fedex_methods = ['Same Day', 'First Overnight', 'Priority Overnight', 'First Overnight',
                 'Priority Overnight', 'Standard Overnight', '2Day AM', '2Day', 'Express Saver',
                 'Ground','Home Delivery','Smartpost']

# Standardized shipping methods based primarily on what is selectable through the API here:
# https://wwwapps.ups.com/ctc/request?loc=en_US. 'Surepost' and 'Standard' are not selectable.
ups_methods = ['Next Day Air Early', 'Next Day Air', 'Next Day Air Saver', '2nd Day Air A.M.',
               '2nd Day Air', '3 Day Select', 'Ground', 'Surepost', 'Standard']

# Standardized state names and codes of the 48 contiguous states based upon USPS standards found here:
# https://www.ups.com/worldshiphelp/WS14/ENU/AppHelp/Codes/State_Province_Codes.htm
state_codes = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 
               'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH',
               'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT',
               'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

pkl_merged = open("test_merged.pickle","wb")

In [85]:
def query(lower,upper):
    """
    Queries for and returns a sample of records that meet where clause criteria
    """
    # initializes query based upon lower and upper data inputs
    sql_query = """
    select * from 
    (select * from 
    (select * from 
    (select year_week, business_sid, upper(trim(industry)) as industry, upper(trim(sub_industry)) as sub_industry,shipper,
    trim(service_type_description) as service_type,package_count, weight,shipment_date,delivery_date, delivery_time, 
    freight_charges,freight_discount_amount,misc_charges,misc_discount_amount, 
    net_charge_amount, zone, upper(trim(sender_city)) as sender_city, upper(trim(sender_state)) as sender_state,
    left(sender_zip,5) as sender_zip, upper(trim(recipient_city)) as recipient_city,
    upper(trim(recipient_state)) as recipient_state, left(recipient_zip,5) as recipient_zip 
    from libras.shipment_details 
    where sender_country = 'US' and recipient_country = 'US' and year_week >= {} and year_week < {}
    and delivery_date is not null
    ) t1 
    where t1.shipment_date is not null) t2 
    where t2.freight_charges > 0) t3 
    where t3.zone is not null or trim(zone)!='' 
    """.format(lower,upper)
    
    # queries database and samples results
    records = pd.read_sql_query(sql_query, MySQLDB).sample(frac = 0.14, replace = False) 
    
    return records

In [86]:
def preprocess(records):
    """
    Preprocesses records to satisfy common cleansing requirements between benchmarking and delivery prediction solutions
    """
    # sets datatypes and standardizes zones to single digits
    records = records[records.zone.apply(lambda x: x.isnumeric())]
    records = records.astype({'freight_charges':'float64',
                              'freight_discount_amount':'float64',
                              'misc_charges':'float64',
                              'misc_discount_amount':'float64',
                              'net_charge_amount':'float64',
                              'zone':'int64'})
    records.zone %= 10
    records = records.astype({'zone':'str'})
    
    # Applies the 'fuzz.partial_ratio' fuzzy macthing algorithm to each record based upon the record's service_type.
    # The partial_ratio function is designed to return the shipping method with the highest score as a two pair tuple
    records_fuzzy_match = []
    for row in records[['shipper','service_type']].itertuples():
        if row.shipper == 'fedex':
            records_fuzzy_match.append(process.extractOne(
                row.service_type,
                fedex_methods,
                scorer = fuzz.partial_ratio))
        else:
            records_fuzzy_match.append(process.extractOne(
                row.service_type,
                ups_methods,
                scorer = fuzz.partial_ratio))
     
    # adds the two-element tuple results of the fuzzy match as additional columns and filters out all records
    # with a score less than 70. Drops the score column after filter is applied.
    records.insert(6, 'std_service_type', [method for method, score in records_fuzzy_match])
    records = records.assign(std_service_type_score = [score for method, score in records_fuzzy_match])
    records = records[records.std_service_type_score >= 70]
    records = records.drop('std_service_type_score', axis=1)
    
    # removes records with sender or recipient states residing outside of the 48 contiguous states
    records[(~records.recipient_state.isin(state_codes)) & (records.recipient_state != '')]
    records[(~records.sender_state.isin(state_codes)) & (records.sender_state != '')]
    
    return records

In [87]:
def store(monthly_records_df):
    """
    Stores results from each query into a pickle
    """
    try:
        temp_df = pd.DataFrame()
        newDF = pd.read_pickle("test_merged.pickle")
        print("Merged pickle contains", len(newDF.index), "records before merge")
        temp_df = newDF.append(monthly_records_df)
        temp_df.to_pickle("test_merged.pickle")
        print("Merged pickle contains", len(temp_df.index), "records after merge")
    except EOFError:
        monthly_records_df.to_pickle("test_merged.pickle")
        print("Merged pickle contains", len(monthly_records_df.index), "records with initial merge")

In [88]:
def process_batch(start_time_, batch_num_, lower_date_range_, upper_date_range_):
    batch_start_time = time.time()
    print("Initiating batch ({}): year_week {} and {} (exclusive)".format(
        batch_num_, lower_date_range_, upper_date_range_))
    
    # queries for a batch of records within a given 4-week range
    print("Total time: {} min | Batch time: {} sec | Querying records".format(
        int(round((time.time() - start_time_)/60, 2)),
        int(round(time.time() - batch_start_time))))
    queried_results = query(lower = lower_date_range_, upper = upper_date_range_)
    
    # preprocesses the record batch
    print("Total time: {} min | Batch time: {} sec | Preprocessing records".format(
        int(round((time.time() - start_time_)/60, 2)),
        int(round(time.time() - batch_start_time))))
    preprocessed_results = preprocess(queried_results)
    
    # appends the record batch
    print("Total time: {} min | Batch time: {} sec | Storing records".format(
        int(round((time.time() - start_time_)/60, 2)),
        int(round(time.time() - batch_start_time))))
    store(preprocessed_results)
    
    # reports final time
    print("Total time: {} min | Batch time: {} sec | Records stored".format(
        int(round((time.time() - start_time_)/60, 2)),
        int(round(time.time() - batch_start_time))))
    print("========================================")

In [89]:
start_time = time.time()
batch_num = 1

#Get records for 2018
min_date = 201823
max_date = 201852 #201852
lower_date_range = min_date
upper_date_range = lower_date_range + 4
while upper_date_range <=max_date:
    if upper_date_range <=201852:
        upper_date_range = lower_date_range + 4
        process_batch(start_time, batch_num, lower_date_range, upper_date_range)
        lower_date_range=upper_date_range
        batch_num += 1

#Get records for 2019        
min_date = 201901
max_date = 201925 #201925
lower_date_range = min_date
upper_date_range = lower_date_range + 4
while upper_date_range <=max_date:
    if upper_date_range <=201925:
        upper_date_range = lower_date_range + 4
        process_batch(start_time, batch_num, lower_date_range, upper_date_range)
        lower_date_range=upper_date_range
        batch_num += 1

Initiating batch (1): year_week 201823 and 201827 (exclusive)
Total time: 0 min | Batch time: 0 sec | Querying records
Total time: 1 min | Batch time: 69 sec | Preprocessing records
Total time: 1 min | Batch time: 107 sec | Storing records
Merged pickle contains 227240 records with initial merge
Total time: 1 min | Batch time: 109 sec | Records stored
Initiating batch (2): year_week 201827 and 201831 (exclusive)
Total time: 1 min | Batch time: 0 sec | Querying records
Total time: 3 min | Batch time: 75 sec | Preprocessing records
Total time: 3 min | Batch time: 110 sec | Storing records
Merged pickle contains 227240 records before merge
Merged pickle contains 432291 records after merge
Total time: 3 min | Batch time: 113 sec | Records stored
Initiating batch (3): year_week 201831 and 201835 (exclusive)
Total time: 3 min | Batch time: 0 sec | Querying records
Total time: 5 min | Batch time: 90 sec | Preprocessing records
Total time: 5 min | Batch time: 129 sec | Storing records
Merged p

In [90]:
pkl_merged.close()

In [91]:
pd.read_pickle('test_merged.pickle').sample(frac=0.25, replace=False).to_pickle('test_merged_sample.pickle')